In [3]:
import torch
import numpy as np
from GPEnv import GPEnv
from torch.utils.data import Dataset
from itertools import count
import torch.nn.functional as F
from torch.distributions import Normal



In [ ]:
env = GPEnv({})


In [ ]:
class MyDataset(Dataset):
    def __init__(self, data):
        super(MyDataset, self).__init__()
        self.data = data
    
    def __getitem__(self, idx):
        return self.data[idx]

In [ ]:
def make_dataset(num_episodes, max_iter, model, env, var):
    data = torch.empty((num_episodes, max_iter, 2, 3), dtype=torch.float)
    for i in range(num_episodes):
        obs = env.reset()
        data[i, 0, 0] = torch.tensor(obs + [0])
        data[i, 0, 1] = torch.tensor(obs + [0])
        hidden = None#model.get_initial_state()
        for k in range(1, max_iter):
            with torch.no_grad():
                out, _, hidden = model([obs], hidden)
            action = Normal(out, var).sample()
            obs, r, _, _ = env.step(action)
            obst, rt, _, _ = env.step(out, shadow=True)
            data[i, k, 0] = torch.tensor(obs + [r])
            data[i, k, 1] = torch.tensor(obst + [rt])
    return data


In [ ]:
#value+reward of all states, real and shadow
#compare real and shadow for advantage
#
def train_episode(data, idx, model, max_iter, gamma, var):
    hidden = None
    for k in range(max_iter):
            act, v, hidden = model(data[idx, k, :, :2], hidden)
            v = v.squeeze()
            hidden = (hidden[0][:,0], hidden[1][:,0])
            vpr = gamma * v + data[idx, k, :, 2]
            if k > 0: #last iteration?
                F.mse_loss(v_old, vpr[1]).backward()
                adv = vpr[0] - vpr[1]
                loss = adv * torch.exp(Normal(act_old, var).log_prob(data[idx,k,0,0]))
                loss.backward()
            v_old = v[0]
            act_old = act[0]


In [17]:
Normal(0, .01).sample()

tensor(-0.0122)

In [10]:
loc = torch.zeros((1,), requires_grad=True)
Normal(loc, 1).rsample()


tensor([-0.9115], grad_fn=<AddBackward0>)

In [9]:

torch.tensor([1,2])[[1]]

tensor([2])

In [13]:
a = [2,3]

In [14]:
a.append(3)

In [28]:
([1,2],[3,4])[:][0]

[1, 2]

In [ ]:



def make_dataset(num_episodes, max_iter, model, env, var):
    data = torch.empty((num_episodes, max_iter, 3), dtype=torch.float)
    for i in range(num_episodes):
        obs = env.reset()
        data[i, 0] = torch.tensor(obs + [0])
        hidden = model.get_initial_state()
        for k in range(1, max_iter):
            with torch.no_grad():
                out, hidden = model(obs, hidden)
            action = Normal(out, var).sample()
            obs, r, t, _ = env.step(action)
            data[i, k] = torch.tensor(obs + [r])
    return data